### make an object of image

In [61]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

#read your file
file='./content/5.png'
img = cv2.imread(file,0)

### To Detect Biggest BOX

In [62]:
# to detect biggest box

# crop image for proper output
h1 = img.shape[0]
w1 = img.shape[1]



#thresholding the image to a binary image
thresh,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

#inverting the image 
img_bin = 255-img_bin


# countcol(width) of kernel as 100th of total width
kernel_len = np.array(img).shape[1]//100
# Defining a vertical kernel to detect all vertical lines of image 
ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
# Defining a horizontal kernel to detect all horizontal lines of image
hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))
# A kernel of 2x2
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

#Use vertical kernel to detect and save the vertical lines in a jpg
image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

#Use horizontal kernel to detect and save the horizontal lines in a jpg
image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

# Combine horizontal and vertical lines in a new third image, with both having same weight.
img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
#Eroding and thesholding the image
img_vh = cv2.erode(~img_vh, kernel, iterations=2)
thresh, img_vh = cv2.threshold(img_vh,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# cv2.imwrite("./content/img_vh.jpg", img_vh)Meetd90168
bitxor = cv2.bitwise_xor(img,img_vh)
bitnot = cv2.bitwise_not(bitxor)

# Detect contours for following box detection
contours, hierarchy = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

# Sort all the contours by top to bottom.
contours, boundingBoxes = sort_contours(contours, method="top-to-bottom")

#Creating a list of heights for all detected boxes
heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]

#Get mean of heights
mean = np.mean(heights)

#Create list box to store all boxes in  
# box = []
bigBox = []
# Get position (x,y), width and height for every contour and show the contour on image
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    if (w>200 and h>200):
        # image = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1)
        bigBox.append([x,y,w,h])
        # cv2.imshow('Image', image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
    
bigBox = bigBox[-1]

In [63]:
providedCropedImage = False
if (img.shape[0] - bigBox[-1]) < 10 :
    providedCropedImage = True # the image is already croped so it has same size of cell so we have to run special code

In [64]:
bigBox

[5, 1, 834, 725]

### crop image

In [65]:
x,y,xe,ye = bigBox
xe = xe+x
ye = ye+y

h = img.shape[0]
w = img.shape[1]
image = img[y+27+2:ye,x+123:xe-2]

# cv2.imshow('Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


## detect boxes and get size of ractangle which we have to create

In [66]:
img = image.copy()
#thresholding the image to a binary image
thresh,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

#inverting the image 
img_bin = 255-img_bin


# countcol(width) of kernel as 100th of total width
kernel_len = np.array(img).shape[1]//100
# Defining a vertical kernel to detect all vertical lines of image 
ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
# Defining a horizontal kernel to detect all horizontal lines of image
hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))
# A kernel of 2x2
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

#Use vertical kernel to detect and save the vertical lines in a jpg
image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

#Use horizontal kernel to detect and save the horizontal lines in a jpg
image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

# Combine horizontal and vertical lines in a new third image, with both having same weight.
img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
#Eroding and thesholding the image
img_vh = cv2.erode(~img_vh, kernel, iterations=2)
thresh, img_vh = cv2.threshold(img_vh,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# cv2.imwrite("./content/img_vh.jpg", img_vh)Meetd90168
bitxor = cv2.bitwise_xor(img,img_vh)
bitnot = cv2.bitwise_not(bitxor)

# Detect contours for following box detection
contours, hierarchy = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

# Sort all the contours by top to bottom.
contours, boundingBoxes = sort_contours(contours, method="top-to-bottom")

#Creating a list of heights for all detected boxes
heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]

#Get mean of heights
mean = np.mean(heights)

#Create list box to store all boxes in  
box = []
flag = False
countCols = 2
firstcolcheck=True
blackBox = []
# Get position (x,y), width and height for every contour and show the contour on image
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    if (w<1000 and h<500):
        # image = cv2.rectangle(img,(x-3,y-3),(x+w,y+h),(0,255,0),2) # did -3 cause new ractangle is not positioned perfectly. 
        # make sure to do the same whenever we use x and y
        box.append([x,y,w,h])

        cellWidth = box[-1][2]
        cellHeight= box[-1][3]

        if len(box)>2:
            px = box[-2][0]
            if firstcolcheck :
                if box[-2][2] == box[-1][2]:
                    countCols += 1
                else:
                    countCols = (3*countCols +1)
                    COUNTCOLS = countCols

                    firstcolcheck = False
            if not firstcolcheck:
                countCols -= 1 

            # print(x,y,w,h)
            # print(countCols)
            xdiff = px - x
            if (x+box[-2][2]-5)<px<(x+box[2][2]+5):
                pass
            elif (x+w) > 10 & xdiff != 104 and xdiff>10 and x>10 and px >10 and xdiff>10:
                if countCols < 0 and box[-1][2] == (box[-2][2]+3):
                    print('differ found....')
                    leftPoint = (x+w)
                    rightPoint = px - 4 - 3 # in some images it gives me errors
                    distanceFromTop = y - 1
                    cellW = box[-2][2] + 3 +3

                    # print(leftPoint,rightPoint,distanceFromTop)
                else:
                    leftPoint = (x+w)
                    rightPoint = px - 4
                    distanceFromTop = y - 1
                    cellW = box[-2][2] + 3

                # print(f"blank dim are : {leftPoint},{rightPoint} y={distanceFromTop}")
                cellH = box[-2][3] + 3
                xs = leftPoint
                rxe = rightPoint
                ys = distanceFromTop
                blackBox.append((xs,rxe,ys,cellW,cellH))
                print(blackBox[-1])
            if countCols < 0 and box[-1][2]>75:
                break
    # cv2.imshow('Image', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()



(288, 651, 58, 52, 22)


In [67]:
blackBox

[(288, 651, 58, 52, 22)]

## draw white box and then draw all the small boxes

In [68]:

def drawrectangles(img,xs,ys,xe,ye,rxe,rye,rows,columns,cellW,cellH):
    firstRow = True
    tempys = ys
    img = cv2.rectangle(img, (xs+1,ys-1), (rxe,rye), (255,255,255),-1 ) # to create big white box in place of black box
    xs = xs
    ys = ys - 2 
    ye = ye - 2 # to create perfect square
    
    flag = False
    orgys = ys
    # print(rows,columns)



    if providedCropedImage:
        for i in range(rows):
            nxs = xs + (i)*cellW
            nxe = xe + i*cellW
            ys = orgys
            for j in range(columns):
                nys = ys + j*cellH
                img = cv2.rectangle(img, (nxs,nys), (nxe,ye), (0,0,0), 1)

    else:

        for i in range(rows):
            nxs = xs + (i)*cellW
            nxe = xe + i*cellW
            ys = orgys
            if flag:
                ys = orgys
            for j in range(columns):
                if j%5 == 0 and j!=0:
                    ys = ys-1
                    flag = True
                nys = ys + j*cellH
                img = cv2.rectangle(img, (nxs,nys), (nxe,ye), (0,0,0), 1)









                # flag = False
                # nys = ys + j*cellH
                # if j%4 == 0 and j != 0:
                #     nye = ye + j*cellH - 1
                #     flag = True
                # else:
                #     nye = ye + j*cellH
                # if j%5 == 0 and j!=0 :
                #     ys = ys - 1
                #     nys = ys + j*cellH 
                #     nye = ye + j*cellH -1

                # print(xs,nys,xe,nye)
                # img = cv2.rectangle(img, (xs,nys), (xe,nye), (0,0,0), 1)

        # for i in range(rows):
        #     if firstRow:
        #         firstRow = False
        #     else:
        #         xs = xe
        #         xe = xe + 52
        #         ys = tempys

        #     firstColumn = True
        #     for j in range(columns+1):
        #         if firstColumn:
        #             firstColumn =False
        #         else:
        #             if j == 15:
        #                 ys = ys + 22 - 3
        #             else:
        #                 ys = ys + 22

        #         img = cv2.rectangle(img, (xs,ys), (xe,ye), (0,0,0), 1)
            # break
        # cv2.imshow('Rectangle',img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
    return img

In [69]:
for box in blackBox:
    xs,rxe,ys,cellW,cellH = box
    # print(xs,rxe,ys,cellW,cellH)
    h,w = img.shape
    rye = h - 4

    xe = xs+(cellW)
    ye = ys+(cellH)
    # print(cellW,cellH)
    rows = (rxe-xs)//(cellW-1)
    columns = (rye-ys)//(cellH-1)

    # print(xe,ye)

    img = drawrectangles(img,xs,ys,xe,ye,rxe,rye,rows,columns,cellW,cellH)
    # cv2.imshow('Rectangle',img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    cv2.imwrite('temp.png',img)

In [70]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

#read your file
file='temp.png'
img = cv2.imread(file,0)
img.shape

#thresholding the image to a binary image
thresh,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

#inverting the image 
img_bin = 255-img_bin
cv2.imwrite('./content/cv_inverted.png',img_bin)
#Plotting the image to see the output
# plotting = plt.imshow(img_bin,cmap='gray')
# plt.show()

# countcol(width) of kernel as 100th of total width
kernel_len = np.array(img).shape[1]//100
# Defining a vertical kernel to detect all vertical lines of image 
ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
# Defining a horizontal kernel to detect all horizontal lines of image
hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))
# A kernel of 2x2
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

#Use vertical kernel to detect and save the vertical lines in a jpg
image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)
cv2.imwrite("./content/vertical.jpg",vertical_lines)
#Plot the generated image
# plotting = plt.imshow(image_1,cmap='gray')
# plt.show()

#Use horizontal kernel to detect and save the horizontal lines in a jpg
image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)
cv2.imwrite("./content/horizontal.jpg",horizontal_lines)
#Plot the generated image
# plotting = plt.imshow(image_2,cmap='gray')
# plt.show()

# Combine horizontal and vertical lines in a new third image, with both having same weight.
img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
#Eroding and thesholding the image
img_vh = cv2.erode(~img_vh, kernel, iterations=2)
thresh, img_vh = cv2.threshold(img_vh,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2.imwrite("/Users/marius/Desktop/img_vh.jpg", img_vh)
bitxor = cv2.bitwise_xor(img,img_vh)
bitnot = cv2.bitwise_not(bitxor)
# Plotting the generated image
# plotting = plt.imshow(bitnot,cmap='gray')
# plt.show()

# Detect contours for following box detection
contours, hierarchy = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

# Sort all the contours by top to bottom.
contours, boundingBoxes = sort_contours(contours, method="top-to-bottom")

#Creating a list of heights for all detected boxes
heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]

#Get mean of heights
mean = np.mean(heights)

#Create list box to store all boxes in  
box = []
# Get position (x,y), width and height for every contour and show the contour on image
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    if (w<1000 and h<500):
        image = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        box.append([x,y,w,h])
        
# plotting = plt.imshow(image,cmap='gray')
# plt.show()

#Creating two lists to define row and column in which cell is located
row=[]
column=[]
j=0

#Sorting the boxes to their respective row and column
for i in range(len(box)):    
        
    if(i==0):
        column.append(box[i])
        previous=box[i]    
    
    else:
        if(box[i][1]<=previous[1]+mean/2):
            column.append(box[i])
            previous=box[i]            
            
            if(i==len(box)-1):
                row.append(column)        
            
        else:
            row.append(column)
            column=[]
            previous = box[i]
            column.append(box[i])
            
# print(column)
# print(row)

#calculating maximum number of cells
countcol = 0
for i in range(len(row)):
    countcol = len(row[i])
    if countcol > countcol:
        countcol = countcol

#Retrieving the center of each column
center = [int(row[i][j][0]+row[i][j][2]/2) for j in range(len(row[i])) if row[0]]

center=np.array(center)
center.sort()
print(center)
#Regarding the distance to the columns center, the boxes are arranged in respective order

finalboxes = []
for i in range(len(row)):
    lis=[]
    for k in range(countcol):
        lis.append([])
    for j in range(len(row[i])):
        diff = abs(center-(row[i][j][0]+row[i][j][2]/4))
        minimum = min(diff)
        indexing = list(diff).index(minimum)
        lis[indexing].append(row[i][j])
    finalboxes.append(lis)


#from every single image-based cell/box the strings are extracted via pytesseract and stored in a list
outer=[]
for i in range(len(finalboxes)):
    for j in range(len(finalboxes[i])):
        inner=''
        if(len(finalboxes[i][j])==0):
            outer.append(' ')
        else:
            for k in range(len(finalboxes[i][j])):
                y,x,w,h = finalboxes[i][j][k][0],finalboxes[i][j][k][1], finalboxes[i][j][k][2],finalboxes[i][j][k][3]
                finalimg = bitnot[x:x+h, y:y+w]
                kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1))
                border = cv2.copyMakeBorder(finalimg,2,2,2,2, cv2.BORDER_CONSTANT,value=[255,255])
                resizing = cv2.resize(border, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
                dilation = cv2.dilate(resizing, kernel,iterations=1)
                erosion = cv2.erode(dilation, kernel,iterations=2)
                
                out = pytesseract.image_to_string(erosion,config='-l eng --oem 3 --psm 10 --dpi 300')
                if(len(out)==0):
                    out = pytesseract.image_to_string(erosion,config='-l eng --oem 3 --psm 10 --dpi 300')
                inner = inner +" "+ out
            outer.append(inner)
# print(outer)


for i in range(len(outer)):
    if outer[i] == ' ist\n\x0c':
        outer[i] = ' 1st\n\x0c'
    if outer[i] == ' 20740"\n\x0c':
        outer[i] = ' 20\"/40\"\n\x0c'
    if outer[i] == ' xX\n\x0c':
        outer[i] = ' X\n\x0c'
    if outer[i] == ' Oo\n\x0c':
        outer[i] = ' \n\x0c'

print(outer)


# #Creating a dataframe of the generated OCR list
# arr = np.array(outer)
# dataframe = pd.DataFrame(arr.reshape(len(row), countcol))
# print(dataframe)
# data = dataframe.style.set_properties(align="left")
# #Converting it in a excel-file
# data.to_excel("/Users/marius/Desktop/output.xlsx")


#Creating a dataframe of the generated OCR list
arr = np.array(outer)
df = pd.DataFrame(arr.reshape(len(row), countcol))
# print(dataframe)
# data = dataframe.style.set_properties(align="left")
#Converting it in a excel-file
# data.to_excel("output.xlsx")
# df=df.replace('Oo','')
# df=df.replace('ist','1st')
# df=df.replace('xX','X')
file_name = '5.csv'
df.to_csv(file_name)

import pandas as pd
# read an excel file and convert 
# into a dataframe object
df = pd.DataFrame(pd.read_csv(file_name))





        

[ 40 107 159 211 263 315 367 419 471 523 575 627 679]
[' SIZE/TYPE\n\x0c', ' 12/14/2022\n\x0c', ' ', ' 12/15/2022\n\x0c', ' ', ' 12/16/2022\n\x0c', ' ', ' 12/17/2022\n\x0c', ' ', ' 12/18/2022\n\x0c', ' ', ' 12/19/2022\n\x0c', ' ', ' ', ' Wednesday\n\x0c', ' ', ' Thursday\n\x0c', ' ', ' Friday\n\x0c', ' ', ' Saturday\n\x0c', ' ', ' Sunday\n\x0c', ' ', ' Monday\n\x0c', ' ', ' ', ' 1st\n\x0c', ' 2nd\n\x0c', ' 1st\n\x0c', ' 2nd\n\x0c', ' 1st\n\x0c', ' 2nd\n\x0c', ' 1st\n\x0c', ' 2nd\n\x0c', ' 1st\n\x0c', ' 2nd\n\x0c', ' 1st\n\x0c', ' 2nd\n\x0c', ' 20"/40"\n\x0c', ' DUAL\n\x0c', ' DUAL\n\x0c', ' DUAL\n\x0c', ' SINGLE\n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' SINGLE\n\x0c', ' 45\n\x0c', ' DUAL\n\x0c', ' DUAL\n\x0c', ' DUAL\n\x0c', ' SINGLE\n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' SINGLE\n\x0c', ' Reefer\n\x0c', ' X\n\x0c', ' X\n\x0c', ' X\n\x0c', ' X\n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', ' \n\x0c', 

In [71]:
# df = pd.DataFrame(pd.read_csv("test.csv"))
# df =df.replace('Oo\n','\n')
# df.iloc[:,5] = df.iloc[:,5]

# df = df.convert_dtypes()
# df.info()
# df=df.replace('xX','X')
# df

In [72]:
outer

[' SIZE/TYPE\n\x0c',
 ' 12/14/2022\n\x0c',
 ' ',
 ' 12/15/2022\n\x0c',
 ' ',
 ' 12/16/2022\n\x0c',
 ' ',
 ' 12/17/2022\n\x0c',
 ' ',
 ' 12/18/2022\n\x0c',
 ' ',
 ' 12/19/2022\n\x0c',
 ' ',
 ' ',
 ' Wednesday\n\x0c',
 ' ',
 ' Thursday\n\x0c',
 ' ',
 ' Friday\n\x0c',
 ' ',
 ' Saturday\n\x0c',
 ' ',
 ' Sunday\n\x0c',
 ' ',
 ' Monday\n\x0c',
 ' ',
 ' ',
 ' 1st\n\x0c',
 ' 2nd\n\x0c',
 ' 1st\n\x0c',
 ' 2nd\n\x0c',
 ' 1st\n\x0c',
 ' 2nd\n\x0c',
 ' 1st\n\x0c',
 ' 2nd\n\x0c',
 ' 1st\n\x0c',
 ' 2nd\n\x0c',
 ' 1st\n\x0c',
 ' 2nd\n\x0c',
 ' 20"/40"\n\x0c',
 ' DUAL\n\x0c',
 ' DUAL\n\x0c',
 ' DUAL\n\x0c',
 ' SINGLE\n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' SINGLE\n\x0c',
 ' 45\n\x0c',
 ' DUAL\n\x0c',
 ' DUAL\n\x0c',
 ' DUAL\n\x0c',
 ' SINGLE\n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' SINGLE\n\x0c',
 ' Reefer\n\x0c',
 ' X\n\x0c',
 ' X\n\x0c',
 ' X\n\x0c',
 ' X\n\x0c',
 ' \n\x0c',
 ' \n\x0c',
 ' \n